## Generates Fake Employee Data - Using Random and Faker, User can choose number of rows

In [ ]:
import random
from faker import Faker
import mysql.connector

# Define the companies, designations, departments, degrees and specializations
companies = ["Amazon", "Facebook", "Myntra", "Qualitest", "Concentrix", "Convergies", "Accenture", "Tata", "TechMahindra", "CocaCola", "Pepsi_Co", "Google"]
degrees = ["Bachelor of Science", "Bachelor of Arts", "Bachelor of Commerce","Bachelor of Technology" ]
specializations = {
    "Bachelor of Science": ["Computer Science", "Mathematics", "Statistics"],
    "Bachelor of Arts": ["Economics", "Management"],
    "Bachelor of Commerce": ["Economics", "Management", "Statistics"],
    "Bachelor of Technology": ["Computer Science"]
}
designations = ["Manager", "Senior Manager", "Team Lead", "Project Lead", "Developer", "Senior Developer", "Data Scientist", "Sr_DataScientist", "DataAnalyst", "Executive"]
departments = ["HR", "Marketing", "Testing", "Developer", "Operations", "L&D"]

# Connect to the database
db = mysql.connector.connect(
    host="enterhost",
    user="enterSQLusername",
    password="enterpassword",
    database="enterdatabase"
)

# Create a cursor
cursor = db.cursor()

# Drop the employee table if it exists
cursor.execute("DROP TABLE IF EXISTS employee")

# Create the employee table
cursor.execute("""
CREATE TABLE employee (
    idx INT NOT NULL AUTO_INCREMENT,
    name VARCHAR(255),
    emp_id VARCHAR(255) NOT NULL PRIMARY KEY,
    department VARCHAR(255),
    designation VARCHAR(255),
    company_name VARCHAR(255),
    degree VARCHAR(255),
    specialization VARCHAR(255),
    salary DECIMAL(10,2),
    tenure INT,
    ess_2019 DECIMAL(3,2),
    ess_2020 DECIMAL(3,2),
    ess_2021 DECIMAL(3,2),
    ess_2022 DECIMAL(3,2),
    INDEX(idx)
)
""")

# Function to insert employee data into the database
def insert_employee_data(num_rows):
    fake = Faker('en_IN')
    for i in range(num_rows):
        emp_id = 'E' + str(random.randint(10000, 99999))
        while True:
            # Check if the generated employee ID already exists in the database
            cursor.execute("SELECT emp_id FROM employee WHERE emp_id = %s", (emp_id,))
            result = cursor.fetchone()
            if result is None:
                # If the employee ID does not exist, break out of the loop
                break
            # If the employee ID already exists, generate a new one
            emp_id = 'E' + str(random.randint(10000, 99999))
        name = fake.name()
        department = random.choice(departments)
        designation = random.choice(designations)
        company_name = random.choice(companies)
        degree = random.choice(degrees)
        
        if degree == "Bachelor of Technology":
            specialization = random.choice(specializations[degree])
            designation = random.choice(["Developer", "Senior Developer"])
            department = random.choice(["Testing", "Developer", "Operations"])
        else:
            specialization = random.choice(specializations[degree])
            department = random.choice(departments)
            
        salary = round(random.uniform(20000, 150000), 2)
        tenure = random.randint(1, 10)
        ess_2019 = round(random.uniform(3, 5), 2)
        ess_2020 = round(random.uniform(3, 5), 2)
        ess_2021 = round(random.uniform(3, 5), 2)
        ess_2022 = round(random.uniform(3, 5), 2)
        query = "INSERT INTO employee (name, emp_id, department, designation, company_name, degree, specialization, salary, tenure, ess_2019, ess_2020, ess_2021, ess_2022) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (name, emp_id, department, designation, company_name, degree, specialization, salary, tenure, ess_2019, ess_2020, ess_2021, ess_2022)
        cursor.execute(query, values)

# Get user input for number of rows to insert
num_rows = int(input("Enter the number of rows of employee data to insert: "))

# Insert employee data in batches of 1000 rows at a time
num_iterations = num_rows // 1000
if num_rows % 1000 != 0:
    num_iterations += 1

for i in range(num_iterations):
    insert_employee_data(min(1000, num_rows))
    num_rows -= 1000

# Commit the changes
db.commit()

# Close the connection
db.close()